# 🤖 Project 3: The Hard-Coded Agent

**Objective:** Build an AI Agent loop from scratch **without frameworks**.

## 📖 What You'll Learn

- The ReAct pattern (Reasoning + Acting)
- How to parse LLM outputs into structured actions
- Building agent loops with Python
- Tool execution and observation feedback

## 🎯 The ReAct Pattern

```
Loop:
  1. THOUGHT: Reason about what to do
  2. ACTION: Call a tool/function
  3. OBSERVATION: Get the result
  4. Repeat until task is complete
```

This is the foundation of **every AI agent framework** (LangChain, AutoGPT, etc.)!

## Setup and Imports

In [ ]:
# Install required packages
# !pip install openai python-dotenv

In [ ]:
import os
import re
import json
from typing import Dict, Any, Optional
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("✅ Setup complete!")

## Task 1: Define Tools (Functions)

Our agent will have access to two simple tools.

In [ ]:
def get_weather(city: str) -> Dict[str, Any]:
    """
    Simulated weather API.
    In production, this would call a real weather service.
    """
    # Mock data
    weather_data = {
        "london": {"temperature": "15°C", "condition": "Cloudy", "humidity": "75%"},
        "new york": {"temperature": "22°C", "condition": "Sunny", "humidity": "60%"},
        "tokyo": {"temperature": "18°C", "condition": "Rainy", "humidity": "85%"},
        "paris": {"temperature": "17°C", "condition": "Partly Cloudy", "humidity": "70%"},
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        return {
            "success": True,
            "city": city,
            "data": weather_data[city_lower]
        }
    else:
        return {
            "success": False,
            "error": f"Weather data not available for {city}"
        }

def calculator(expression: str) -> Dict[str, Any]:
    """
    Safe calculator that evaluates mathematical expressions.
    """
    try:
        # Only allow basic math operations for security
        allowed_chars = set('0123456789+-*/(). ')
        if not all(c in allowed_chars for c in expression):
            return {
                "success": False,
                "error": "Invalid characters in expression"
            }
        
        result = eval(expression)
        return {
            "success": True,
            "expression": expression,
            "result": result
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

# Test the tools
print("🧪 Testing tools:\n")
print("Weather in London:", get_weather("London"))
print("Calculate 25 * 4:", calculator("25 * 4"))

## Task 2: Design the System Prompt

The prompt is **critical**. It teaches the LLM:
1. What tools it has access to
2. The exact format to use
3. When to stop

In [ ]:
SYSTEM_PROMPT = """You are an AI agent with access to tools. Your job is to answer questions by using these tools.

AVAILABLE TOOLS:
1. get_weather(city: str) - Get current weather for a city
   Example: get_weather("London")

2. calculator(expression: str) - Calculate mathematical expressions
   Example: calculator("25 * 4")

RESPONSE FORMAT:
You must ALWAYS respond in this exact format:

THOUGHT: [Your reasoning about what to do next]
ACTION: [tool_name(arguments)] OR None if ready to answer
OBSERVATION: [Wait for the system to provide this]
ANSWER: [Only provide this when you have enough information]

RULES:
1. Always start with THOUGHT
2. If you need information, specify an ACTION
3. If you have enough information, set ACTION to None and provide ANSWER
4. Never make up tool results - wait for OBSERVATION
5. Keep thoughts concise and focused

EXAMPLE:
Question: What's the weather in Paris?
THOUGHT: I need to check the weather for Paris
ACTION: get_weather("Paris")
OBSERVATION: {"success": true, "data": {"temperature": "17°C", "condition": "Partly Cloudy"}}
THOUGHT: I have the weather information
ACTION: None
ANSWER: The weather in Paris is 17°C and partly cloudy.
"""

print("📋 System Prompt:")
print("="*80)
print(SYSTEM_PROMPT)
print("="*80)

## Task 3: Parse LLM Output

Extract structured actions from the LLM's text response.

In [ ]:
def parse_action(text: str) -> Optional[Dict[str, str]]:
    """
    Parse the ACTION line from LLM response.
    
    Returns:
        Dict with 'tool' and 'args', or None if ACTION is None
    """
    # Find the ACTION line
    action_match = re.search(r'ACTION:\s*(.+)', text)
    if not action_match:
        return None
    
    action_text = action_match.group(1).strip()
    
    # Check if action is None (agent is ready to answer)
    if action_text.lower() == 'none':
        return None
    
    # Parse tool call: tool_name(arguments)
    tool_match = re.match(r'(\w+)\((.*)\)', action_text)
    if not tool_match:
        raise ValueError(f"Invalid action format: {action_text}")
    
    tool_name = tool_match.group(1)
    args = tool_match.group(2).strip('"\' ')  # Remove quotes
    
    return {
        'tool': tool_name,
        'args': args
    }

# Test the parser
test_cases = [
    'ACTION: get_weather("London")',
    'ACTION: calculator("25 * 4")',
    'ACTION: None',
]

print("🧪 Testing action parser:\n")
for test in test_cases:
    result = parse_action(test)
    print(f"Input:  {test}")
    print(f"Output: {result}\n")

## Task 4: Execute Tools

Map tool names to actual Python functions.

In [ ]:
# Tool registry
TOOLS = {
    'get_weather': get_weather,
    'calculator': calculator,
}

def execute_action(action: Dict[str, str]) -> str:
    """
    Execute a tool and return the result as a string.
    """
    tool_name = action['tool']
    args = action['args']
    
    if tool_name not in TOOLS:
        return json.dumps({"success": False, "error": f"Unknown tool: {tool_name}"})
    
    # Execute the tool
    tool_func = TOOLS[tool_name]
    result = tool_func(args)
    
    return json.dumps(result)

# Test execution
test_action = {'tool': 'get_weather', 'args': 'Tokyo'}
print("🧪 Testing tool execution:")
print(f"Action: {test_action}")
print(f"Result: {execute_action(test_action)}")

## Task 5: Build the Agent Loop

Now we connect everything: LLM → Parse → Execute → Feedback → Repeat

In [ ]:
def run_agent(question: str, max_iterations: int = 5, verbose: bool = True) -> str:
    """
    Run the agent loop to answer a question.
    
    Args:
        question: User's question
        max_iterations: Maximum number of tool calls to prevent infinite loops
        verbose: Print step-by-step execution
    
    Returns:
        Final answer from the agent
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]
    
    if verbose:
        print("="*80)
        print(f"🎯 Question: {question}")
        print("="*80 + "\n")
    
    for iteration in range(max_iterations):
        if verbose:
            print(f"🔄 Iteration {iteration + 1}")
            print("-"*80)
        
        # Get LLM response
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0,  # Deterministic for consistent format
        )
        
        assistant_message = response.choices[0].message.content
        
        if verbose:
            print(f"\n🤖 Agent Response:\n{assistant_message}\n")
        
        # Add to conversation history
        messages.append({"role": "assistant", "content": assistant_message})
        
        # Parse action
        try:
            action = parse_action(assistant_message)
        except ValueError as e:
            if verbose:
                print(f"❌ Parse error: {e}")
            return f"Error: {e}"
        
        # Check if agent is done
        if action is None:
            # Extract final answer
            answer_match = re.search(r'ANSWER:\s*(.+)', assistant_message, re.DOTALL)
            if answer_match:
                final_answer = answer_match.group(1).strip()
                if verbose:
                    print("="*80)
                    print("✅ Agent Complete!")
                    print("="*80)
                return final_answer
            else:
                return "Error: Agent finished but provided no answer"
        
        # Execute tool
        if verbose:
            print(f"⚙️  Executing: {action['tool']}({action['args']})")
        
        observation = execute_action(action)
        
        if verbose:
            print(f"👁️  Observation: {observation}\n")
        
        # Add observation to messages
        messages.append({
            "role": "user",
            "content": f"OBSERVATION: {observation}"
        })
    
    return "Error: Maximum iterations reached without answer"

print("✅ Agent loop ready!")

## 🧪 Test the Agent

### Test 1: Simple Weather Query

In [ ]:
answer = run_agent("What's the weather like in Tokyo?")
print(f"\n📝 Final Answer: {answer}")

### Test 2: Math Calculation

In [ ]:
answer = run_agent("If I have 15 apples and buy 7 more, then give away 9, how many do I have?")
print(f"\n📝 Final Answer: {answer}")

### Test 3: Multi-Tool Query

In [ ]:
answer = run_agent(
    "What's the temperature in London and New York? What's the average?"
)
print(f"\n📝 Final Answer: {answer}")

### Test 4: Complex Reasoning

In [ ]:
answer = run_agent(
    "I'm in Paris and want to visit a city that's 5 degrees warmer. Which city should I go to?"
)
print(f"\n📝 Final Answer: {answer}")

## 🎯 Challenge Exercises

### Challenge 1: Add a New Tool

Implement a `currency_converter(amount, from_currency, to_currency)` tool.

In [ ]:
def currency_converter(args: str) -> Dict[str, Any]:
    """
    Convert between currencies.
    Args format: "100 USD to EUR"
    
    TODO: Implement this function
    Hints:
    - Parse the args string to extract amount, from_currency, to_currency
    - Use mock exchange rates
    - Return result in same format as other tools
    """
    # YOUR CODE HERE
    pass

# Don't forget to:
# 1. Add to TOOLS dictionary
# 2. Update SYSTEM_PROMPT with tool description
# 3. Test it!

### Challenge 2: Improve Error Handling

The agent should gracefully handle:
- Invalid tool names
- Malformed arguments
- Tool execution failures

In [ ]:
# TODO: Modify the agent loop to:
# 1. Catch parsing errors and send them back as observations
# 2. Let the LLM try again with corrected input
# 3. Keep track of retry attempts

# YOUR CODE HERE

### Challenge 3: Add Conversation History

Make the agent remember previous questions in the same session.

In [ ]:
class ConversationalAgent:
    """
    Agent that maintains conversation history.
    
    TODO: Implement this class
    Methods:
    - __init__: Initialize with system prompt
    - ask(question): Process a question, return answer
    - reset(): Clear conversation history
    """
    # YOUR CODE HERE
    pass

## 🎓 Key Takeaways

### What You've Learned:

1. **The ReAct Pattern**:
   - Separates reasoning (THOUGHT) from action (ACTION)
   - Grounds LLM outputs in real tool execution
   - Provides feedback loop through OBSERVATION

2. **Prompt Engineering is Critical**:
   - Clear format instructions reduce parsing errors
   - Examples in prompts improve consistency
   - Temperature=0 for structured outputs

3. **Agent Loop Architecture**:
   ```python
   while not done:
       response = llm(messages)
       action = parse(response)
       if action is None: return answer
       result = execute(action)
       messages.append(observation)
   ```

4. **Tool Abstraction**:
   - Tools are just Python functions
   - Consistent interface (input/output format)
   - Error handling is essential

### Why Frameworks Exist:

You just built what LangChain/LangGraph do under the hood! Frameworks add:
- Pre-built tools
- Better error handling
- Streaming responses
- Memory management
- Monitoring/observability

But now you understand **how they work**!

### Production Considerations:

- **Safety**: Validate tool inputs, sandbox execution
- **Cost**: Each iteration = API call
- **Latency**: Multiple round trips add up
- **Reliability**: Need retry logic, timeouts

### Next Steps:

In Module 4, you'll add **memory** (RAG) so the agent can access knowledge bases!

## 📚 Additional Resources

- [ReAct: Synergizing Reasoning and Acting (Paper)](https://arxiv.org/abs/2210.03629)
- [MRKL Systems (Paper)](https://arxiv.org/abs/2205.00445)
- [LangChain Agent Documentation](https://python.langchain.com/docs/modules/agents/)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)